In [18]:
import pandas as pd
import pulp

# Load data
demand = pd.read_csv('dataset/demand.csv')
vehicles = pd.read_csv('dataset/vehicles.csv')
vehicles_fuels = pd.read_csv('dataset/vehicles_fuels.csv')
fuels = pd.read_csv('dataset/fuels.csv')
carbon_emissions = pd.read_csv('dataset/carbon_emissions.csv')

# Initialize optimization problem
prob = pulp.LpProblem("dataset/FleetDecarbonization", pulp.LpMinimize)

# Define decision variables
buy = pulp.LpVariable.dicts("Buy", ((year, vehicle) for year in range(2023, 2039) for vehicle in vehicles['ID']), 0, None, pulp.LpInteger)
use = pulp.LpVariable.dicts("Use", ((year, vehicle) for year in range(2023, 2039) for vehicle in vehicles['ID']), 0, None, pulp.LpInteger)
sell = pulp.LpVariable.dicts("Sell", ((year, vehicle) for year in range(2023, 2039) for vehicle in vehicles['ID']), 0, None, pulp.LpInteger)
distance = pulp.LpVariable.dicts("Distance", ((year, vehicle) for year in range(2023, 2039) for vehicle in vehicles['ID']), 0, None, pulp.LpContinuous)

# Objective function: Minimize total cost
total_cost = pulp.lpSum([
    buy[(year, vehicle)] * vehicles.loc[vehicles['ID'] == vehicle, "Cost ($)"].values[0] for year in range(2023, 2039) for vehicle in vehicles['ID']
])

prob += total_cost

# Constraints
# 1. Demand fulfillment
for year in range(2023, 2039):
    for size in demand['Size'].unique():
        for dist in demand['Demand (km)'].unique():
            prob += (pulp.lpSum([
                use[(year, vehicle)] * distance[(year, vehicle)] for vehicle in vehicles.loc[(vehicles['Size'] == size) & (vehicles['Yearly range (km)'] >= dist), 'ID']
            ]) >= demand.loc[(demand['year'] == year) & (demand['Size'] == size) & (demand['Distance'] == dist), 'value'].values[0])

# 2. Carbon emission limits
for year in range(2023, 2039):
    prob += (pulp.lpSum([
        distance[(year, vehicle)] * vehicles_fuels.loc[vehicles_fuels['vehicle_ID'] == vehicle, 'fuel_consumption'].values[0] * fuels.loc[fuels['fuel_type'] == vehicles_fuels.loc[vehicles_fuels['vehicle_ID'] == vehicle, 'fuel_type'].values[0], 'carbon_emission'].values[0]
        for vehicle in vehicles['ID']
    ]) <= carbon_emissions.loc[carbon_emissions['year'] == year, 'emission_limit'].values[0])

# 3. Vehicle lifespan and annual sales limits
for year in range(2023, 2039):
    for vehicle in vehicles['ID']:
        if year >= 2023 + 10:
            prob += (use[(year, vehicle)] == 0)
        if year > 2023:
            prob += (sell[(year, vehicle)] <= 0.2 * (pulp.lpSum([buy[(y, vehicle)] for y in range(2023, year)]) - pulp.lpSum([sell[(y, vehicle)] for y in range(2023, year)])))

# Solve the problem
prob.solve()

# Generate solution
solution = []
for year in range(2023, 2039):
    for vehicle in vehicles['ID']:
        solution.append([year, vehicle, buy[(year, vehicle)].varValue, 'Buy', '', '', ''])
        solution.append([year, vehicle, use[(year, vehicle)].varValue, 'Use', '', '', distance[(year, vehicle)].varValue])
        solution.append([year, vehicle, sell[(year, vehicle)].varValue, 'Sell', '', '', ''])

solution_df = pd.DataFrame(solution, columns=['Year', 'ID', 'Num_Vehicles', 'Type', 'Fuel', 'Distance_bucket', 'Distance_per_vehicle(km)'])
solution_df.to_csv('solution.csv', index=False)


KeyError: 'year'

In [7]:
vehicles.head()


,ID,Vehicle,Size,Year,Cost ($),Yearly range (km),Distance
0,BEV_S1_2023,BEV,S1,2023,187000,102000,D1
1,BEV_S1_2024,BEV,S1,2024,177650,102000,D1
2,BEV_S1_2025,BEV,S1,2025,168767,102000,D1
3,BEV_S1_2026,BEV,S1,2026,160329,102000,D2
4,BEV_S1_2027,BEV,S1,2027,152312,102000,D2


In [16]:
 prob += (pulp.lpSum([
                use[(year, vehicle)] * distance[(year, vehicle)] for vehicle in vehicles.loc[(vehicles['Size'] == "s1") & (vehicles['Distance'] >= dist), 'ID']
            ]) >= demand.loc[(demand['year'] == year) & (demand['Size'] == size) & (demand['Distance'] == dist), 'value'].values[0])


KeyError: 'Cost($)'

In [19]:
import pandas as pd
import pulp

# Load data
vehicles = pd.read_csv('vehicles.csv')
vehicles_fuels = pd.read_csv('vehicles_fuels.csv')
fuels = pd.read_csv('fuels.csv')
carbon_emissions = pd.read_csv('carbon_emissions.csv')
demand = pd.read_csv('demand.csv')
cost_profiles = pd.read_csv('cost_profiles.csv')

# Initialize optimization problem
prob = pulp.LpProblem("FleetDecarbonization", pulp.LpMinimize)

# Define decision variables
buy = pulp.LpVariable.dicts("Buy", ((year, vehicle) for year in range(2023, 2039) for vehicle in vehicles['ID']), 0, None, pulp.LpInteger)
use = pulp.LpVariable.dicts("Use", ((year, vehicle) for year in range(2023, 2039) for vehicle in vehicles['ID']), 0, None, pulp.LpInteger)
sell = pulp.LpVariable.dicts("Sell", ((year, vehicle) for year in range(2023, 2039) for vehicle in vehicles['ID']), 0, None, pulp.LpInteger)
distance = pulp.LpVariable.dicts("Distance", ((year, vehicle) for year in range(2023, 2039) for vehicle in vehicles['ID']), 0, None, pulp.LpContinuous)


demand = pd.read_csv('dataset/demand.csv')
vehicles = pd.read_csv('dataset/vehicles.csv')
vehicles_fuels = pd.read_csv('dataset/vehicles_fuels.csv')
fuels = pd.read_csv('dataset/fuels.csv')
carbon_emissions = pd.read_csv('dataset/carbon_emissions.csv')

# Initialize optimization problem
prob = pulp.LpProblem("dataset/FleetDecarbonization", pulp.LpMinimize)
# Objective function: Minimize total cost
total_cost = pulp.lpSum([
    buy[(year, vehicle)] * vehicles.loc[vehicles['ID'] == vehicle, 'Cost ($)'].values[0] for year in range(2023, 2039) for vehicle in vehicles['ID']
]) # Add other cost components

prob += total_cost

# Constraints
# 1. Demand fulfillment
for year in range(2023, 2039):
    for size in demand['Size'].unique():
        for dist in demand['Distance'].unique():
            prob += (pulp.lpSum([
                use[(year, vehicle)] * distance[(year, vehicle)] for vehicle in vehicles.loc[(vehicles['Size'] == size) & (vehicles['Distance'] == dist), 'ID']
            ]) >= demand.loc[(demand['Year'] == year) & (demand['Size'] == size) & (demand['Distance'] == dist), 'Demand (km)'].values[0])

# 2. Carbon emission limits
for year in range(2023, 2039):
    prob += (pulp.lpSum([
        distance[(year, vehicle)] * vehicles_fuels.loc[vehicles_fuels['ID'] == vehicle, 'Consumption (unit_fuel/km)'].values[0] * fuels.loc[fuels['Year'] == year, 'Emissions (CO2/unit_fuel)'].values[0]
        for vehicle in vehicles['ID']
    ]) <= carbon_emissions.loc[carbon_emissions['Year'] == year, 'Carbon emission CO2/kg'].values[0])

# 3. Vehicle lifespan and annual sales limits
for year in range(2023, 2039):
    for vehicle in vehicles['ID']:
        if year >= int(vehicle.split('_')[2]) + 10:
            prob += (use[(year, vehicle)] == 0)
        if year > 2023:
            prob += (sell[(year, vehicle)] <= 0.2 * (pulp.lpSum([buy[(y, vehicle)] for y in range(2023, year)]) - pulp.lpSum([sell[(y, vehicle)] for y in range(2023, year)])))

# Solve the problem
prob.solve()

# Generate solution
solution = []
for year in range(2023, 2039):
    for vehicle in vehicles['ID']:
        solution.append([year, vehicle, buy[(year, vehicle)].varValue, 'Buy', '', '', ''])
        solution.append([year, vehicle, use[(year, vehicle)].varValue, 'Use', '', '', distance[(year, vehicle)].varValue])
        solution.append([year, vehicle, sell[(year, vehicle)].varValue, 'Sell', '', '', ''])

solution_df = pd.DataFrame(solution, columns=['Year', 'ID', 'Num_Vehicles', 'Type', 'Fuel', 'Distance_bucket', 'Distance_per_vehicle(km)'])
solution_df.to_csv('predictions.csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'vehicles.csv'